In [95]:
# Problem Domain: 
# Dataset: 

In [96]:
# Research Questions:


# Objectives:


In [97]:
# Importing dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [98]:
# Loading the dataset
# tracks = pd.read_csv('.csv')
# artists = pd.read_csv('.csv')
# Reference: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html#

In [99]:
# Creating a detailed Dataset Description
# print("Dataset Shape:", df.shape)
# print("Dataset Info:")
# print(df.info())
# print("First Five Rows:")
# print(df.head())

In [100]:
# Examine distributions
# for col in df.columns:
#     plt.figure(figsize=(10,4))
#     sns.histplot(df[col], kde=True)
#     plt.title(f'Distribution of {col}')
#     plt.show()

In [101]:
# Check for missing values
# print("Missing Values:")
# print(df.isnull().sum())